In [1]:
import numpy as np
from numpy import linalg as LA
from scipy.stats import chi2, bernoulli, multinomial
from tqdm.auto import tqdm
import math
import matplotlib.pyplot as plt

In [2]:
def sigma_matrix(p_0, sigma, d):
    Id = np.eye(d)
    IId = np.ones((d,d))/d
    Projection_matrix = (Id - IId)
    diag_matrix = np.diag(p_0) + np.eye(d)*sigma
    M = Projection_matrix @ np.linalg.inv(diag_matrix - np.outer(p_0, p_0)) @ Projection_matrix
    return(M)


def noise_norm(x, p, alpha, p_0):
    H = np.sum(x, axis = 0)
    n = len(x)
    dim = len(H)
    D = np.random.normal(0, np.sqrt(len(x)/p), dim)
    H_noise = H + D
    q = n * (H_noise/n - p_0).reshape(1, -1) @ sigma_matrix(p_0, n / p, dim) @ (H_noise/n - p_0).reshape(-1,1)
    if q > chi2(dim-1).ppf(1 - alpha):
        return(1)
    else:
        return(0)

def noise_lap(x, alpha, p_0, eps):
    H = np.sum(x, axis = 0)
    D = np.sum(np.random.laplace(0, 2 / eps, size=(len(x),len(H))), axis = 0)
    H_noise_lap = H + D
    M_s = sigma_matrix(p_0, 8 * len(x)/eps**(2), len(H))
    q = len(x) * (H_noise_lap/len(x) - p_0).reshape(1, -1) @ M_s @ (H_noise_lap/len(x) - p_0).reshape(-1,1)
    ar = []
    for i in range(50):
        x_0 = np.random.multinomial(1, p0, size=len(x))
        H_0 = np.sum(x_0, axis = 0)
        D = np.sum(np.random.laplace(0, 2/eps, size=(len(x),len(H))), axis=0)
        H_noise_lap = H_0 + D
        ar.append(len(x)* (H_noise_lap/len(x) - p_0).reshape(1, -1) @ sigma_matrix(p_0, 8 * len(x)/ eps ** 2, len(H)) @ (H_noise_lap/len(x) - p_0).reshape(-1, 1))   
    ar = np.sort(ar)
    l = len(ar)+1
    v = (1-alpha)
    if q > ar[int((l) * v - 1)]:
        return(1)
    else:
        return(0)
